# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.9.1'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

we first apply transformation on only the images of training set, to avoid overfitting.
The trasnforamtions are geometraical, zooms and or rotation.
Called image augmentation.

In [3]:
train_datagen = ImageDataGenerator(
        rescale=1./255, #feature scaling to each pixel
        shear_range=0.2, #transformation
        zoom_range=0.2, #transformation
        horizontal_flip=True) #transformation
      
training_set = train_datagen.flow_from_directory(
        'dataset/training_set', 
        target_size=(64, 64), #reduced from 150 to 64 to make training faster
        batch_size=32, #how mnay images we want in each batch. 32 is normal
        class_mode='binary') #binary bc only two outcomes
        

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [4]:
test_datagen = ImageDataGenerator(rescale=1./255) 
#dont want to flip or do other transfroms to keep the test set in tact
#however we still need to scale them

test_set = test_datagen.flow_from_directory( #pulls data into the tesst set
        'dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [5]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3])) #to add first convolutional layer
#this layer is an object of the conv 2D class
#conv2d has a feautre detector being the filters. classic is 32
#input shape - shape of input. in this case it is rgb the shape is 64 64 3
# if black and white 64 64 1
#kernel size is 3 bc want a 3 b 3 kernel

### Step 2 - Pooling

In [7]:
#we are going to apply(add) max pooling
#max pooling is declared as an object of a class
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
#paramters of maxpool2d
#first parameter is size of filter (frame) that moves over the image. only needs one input when it is a square
#2nd parameter = stride: num of pixels which frame is shifted right. 
#once you reach the end of the image you can use padding to add zeros for these points. default has valid padding


### Adding a second convolutional layer

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu')) #to add 2nd convolutional layer
#for 2nd layer dont need to specificy the shape
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [9]:
#creates 1d vector which becmoes the input of the future fully cnn
cnn.add(tf.keras.layers.Flatten()) #does not need parameters

### Step 4 - Full Connection

In [10]:
#add a new fully connected layer.
#done using the tensor flow, keras dense class
#first parameter is units= : hidden nuerons in fully connected layer. choose larger number bc more data in this case
#until you reach output layer generally just use relu
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [11]:
#add a new fully connected layer using the dense class
#excpet bc need less neurons in ouput layer, for binary classification it is 1
#use sigmoid activation func at output for binary classification
#if doing multi-class classification we would use softmax
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [12]:
cnn.compile(optimizer= 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [13]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)
#use less epochs cause this takes longer
#how to train on training set whilst evualting on th te test set

Epoch 1/25
250/250 [==============================] - 47s 187ms/step - loss: 0.6434 - accuracy: 0.6109 - val_loss: 0.5569 - val_accuracy: 0.7370
Epoch 2/25
250/250 [==============================] - 42s 167ms/step - loss: 0.5838 - accuracy: 0.6955 - val_loss: 0.5565 - val_accuracy: 0.7160
Epoch 3/25
250/250 [==============================] - 46s 182ms/step - loss: 0.5455 - accuracy: 0.7251 - val_loss: 0.5112 - val_accuracy: 0.7595
Epoch 4/25
250/250 [==============================] - 46s 184ms/step - loss: 0.5133 - accuracy: 0.7459 - val_loss: 0.5413 - val_accuracy: 0.7370
Epoch 5/25
250/250 [==============================] - 46s 182ms/step - loss: 0.4932 - accuracy: 0.7581 - val_loss: 0.5064 - val_accuracy: 0.7570
Epoch 6/25
250/250 [==============================] - 47s 187ms/step - loss: 0.4782 - accuracy: 0.7716 - val_loss: 0.5110 - val_accuracy: 0.7620
Epoch 7/25
250/250 [==============================] - 44s 177ms/step - loss: 0.4723 - accuracy: 0.7781 - val_loss: 0.5492 - val_ac

## Part 4 - Making a single prediction

In [31]:
import numpy as np
from keras.preprocessing import image
import keras.utils as ut

test_image = ut.load_img('dataset/single_prediction/cat_or_dog_4.jpg', target_size = (64, 64)) #load image we want to test our model on
#the size of the image must be resclaed to be the same as what was used for trainging and testing the model

#predict method reuires a 2d array input
test_image = ut.img_to_array(test_image)

test_image = np.expand_dims(test_image, axis=0) #extra dimensoin which corresponds to the batch size
#set axis=0 bc so dimesion of batch we are adding to the image will be the first dimension

#test image now has correct format neede for the predict method

result = cnn.predict(test_image/255.0) #is a batch dimesion
print(result)
if result[0][0]==1: #[batch][element in bacth]. not using >0.5 and not ==1 prvoides greater accuracy
  prediction='dog'
else:
  prediction='cat'

1/1 [==============================] - 0s 49ms/step
[[0.5350223]]


In [32]:
print(prediction)

cat


In [ ]:
training_set.class_indices